In [43]:
import requests
import datetime
from urllib.parse import urlencode
import base64

In [44]:
#client_id = '234d56555aa14d96a811661dfcf00d64'
#client_secret = '0fb68f689a0c4faabbadc9f88571d739'

client_id = '548b1a8fc43140f88b7d651c037f2c14'
client_secret = '242bdead8b654cdfb1054207075e0627'

In [45]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
        
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ): # type
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = "%20".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
        
        
        
    
    
    

In [46]:
spotify = SpotifyAPI(client_id, client_secret)

In [47]:
spotify.search({"track": "A lannister always pays his debts", "artist": "Ramin"}, search_type="track")

q=track%3AA+lannister+always+pays+his+debts%2520artist%3ARamin&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AA+lannister+always+pays+his+debts%2520artist%3ARamin&type=track&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [48]:
spotify.search({"track": "Time", "artist": "Hans"}, search_type="track")

q=track%3ATime%2520artist%3AHans&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ATime%2520artist%3AHans&type=track&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [49]:
spotify.search({"track": "Time"}, search_type="track")

q=track%3ATime&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ATime&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6KImCVD70vtIoJWnq6nGn3'},
       'href': 'https://api.spotify.com/v1/artists/6KImCVD70vtIoJWnq6nGn3',
       'id': '6KImCVD70vtIoJWnq6nGn3',
       'name': 'Harry Styles',
       'type': 'artist',
       'uri': 'spotify:artist:6KImCVD70vtIoJWnq6nGn3'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
  

In [50]:
spotify.search(query="Time", operator='NOT', operator_query='Bilie Ellish', search_type="track")

q=Time+NOT+Bilie+Ellish&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time+NOT+Bilie+Ellish&type=track&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [41]:
spotify.search(query="Danger", operator='NOT', operator_query='Zone', search_type="track")

q=Danger+NOT+Zone&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Danger+NOT+Zone&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4MCBfE4596Uoi2O4DtmEMz'},
       'href': 'https://api.spotify.com/v1/artists/4MCBfE4596Uoi2O4DtmEMz',
       'id': '4MCBfE4596Uoi2O4DtmEMz',
       'name': 'Juice WRLD',
       'type': 'artist',
       'uri': 'spotify:artist:4MCBfE4596Uoi2O4DtmEMz'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
 

In [31]:
spotify.get_artist("1hCkSJcXREhrodeIHQdav8")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
 'followers': {'href': None, 'total': 433063},
 'genres': ['german soundtrack',
  'scorecore',
  'soundtrack',
  'video game music'],
 'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
 'id': '1hCkSJcXREhrodeIHQdav8',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/3192cec3d51144c7a061848927f89983835c5480',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/d33c97f0483a37ecf2d8309f99b1abaa1e499fd9',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/553f6b2aa9cb60ac1353f8efbd9edf3542332e8f',
   'width': 160}],
 'name': 'Ramin Djawadi',
 'popularity': 73,
 'type': 'artist',
 'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}

In [ ]:
spotify.search("Time", search_type="Track")

In [ ]:
spotify.get_album("41zMFsCjcGenYKVJYUXU2n")